# To do:
* compare variable month and years to stata read data to verify reading them correctly
* make github page for this


In [28]:
################### import packages #######################################################
import pandas as pd
import re
import numpy as np
import os
from hrs_functions import *
import copy 
## maybe make this more programatic
## each survey has a letter preceeding variable names, increases by 1 each time
var_dict = {
    2008: 'l'
    , 2010: 'm'
    , 2012: 'n'
    , 2014: 'o'
    , 2016: 'p' 
}

#################################### define functions ####################################
def read_yr_of_data(directory, variables_to_look_for, survey_year):
    # list all files to potentially read
    directory_sta = directory + 'h' + str(survey_year)[2:4] + 'sta'
    directory_da = directory + 'h' + str(survey_year)[2:4] + 'da'
    path_list = list(set([directory_sta + '/' +  x.split('.')[0] for x in os.listdir(directory_sta) 
                          if 'dct' in x
                          and 'R' in x[len(x.split('.')[0])-1]
                         ]))
    
    # figure out which surveys to read based on variables to look for
    surveys = [x[0] for x in variables_to_look_for]\
        + [x[0:2] for x in variables_to_look_for if 'lb' in x]
    # find the surveys in the path list
    path_short_list = [x for x in path_list if 'r' in x.split('/')[8].lower() # only get respondent
                       and any(
                           y == x.split('/')[8].lower()[3:len(x.split('/')[8]) - 2] 
                           for y in surveys
                       ) 
                      ]
    
    # these are the id's for all R level surveys in all years
    id_variables = ['hhid', 'pn']
    
    survey_specific_variables = [var_dict[survey_year] + x
                             for x in variables_to_look_for]
    rename_dict = copy.deepcopy(variables_to_look_for)
    vars_list = [x for x in variables_to_look_for]
    
    for x in range(len(survey_specific_variables)):
        rename_dict[survey_specific_variables[x]] = rename_dict.pop(vars_list[x])
    
    # read the files
    for i in range(len(path_short_list)):
        path = path_short_list[i]
        filename = directory_da + '/' + path.split('/')[8] + '.da'
        dct_file = directory_sta + '/' + path.split('/')[8] +  '.dct'
        print(filename)
        try:
            temp_df = read_stata_files(filename, dct_file)
            temp_df = temp_df[[x for x in temp_df.columns if x.lower() in (id_variables + survey_specific_variables)]]
            temp_df = temp_df.rename(columns = rename_dict)
            ## maybe rename them here? some sort of naming dictionary
            
            if i == 0:
                year_frame = temp_df
            else:
                year_frame = year_frame.merge(temp_df, on = ["hhid", "pn"], how = "outer")
        except:
            print('missing files: ' + path)
            pass
        
    # make the wave variable, 1992 was wave 1
    wave = (survey_year - 1990) / 2
    year_frame['wave'] = wave
    return year_frame

In [29]:
cwd = os.getcwd()
variables_to_look_for = {'a500':'rmonth_survey'
                         , 'a501':'ryear_survey'
                         , 'j073m1':'rjobwhym1'
                         , 'lb032_2':'rrisk_financial'
                         , 'n106':'rhosp_cost_oop'}
survey_year = 2016
directory = cwd + '/data/'

for yr in [2014, 2016]:
    frame = read_yr_of_data(directory, variables_to_look_for, yr)
    print(frame)

/Users/jcoven/Desktop/HRS_Assembly/python/data/h14da/H14A_R.da
/Users/jcoven/Desktop/HRS_Assembly/python/data/h14da/H14L_R.da
/Users/jcoven/Desktop/HRS_Assembly/python/data/h14da/H14J_R.da
/Users/jcoven/Desktop/HRS_Assembly/python/data/h14da/H14LB_R.da
/Users/jcoven/Desktop/HRS_Assembly/python/data/h14da/H14N_R.da
         hhid   pn  rmonth_survey  ryear_survey  rjobwhym1  rrisk_financial  \
0          30  200             52           142        NaN             20.0   
1      100010  100             42           142        NaN              NaN   
2      100030  300             32           142        NaN             40.0   
3      100040  400             42           142        NaN             60.0   
4      100130  401             42           142        NaN             40.0   
5      100130  102             82           142        NaN              NaN   
6      100380  100             42           142        NaN             80.0   
7      100380  400             42           142     

In [38]:
frame.groupby(['rmonth_survey', 'ryear_survey']).count()

filename = '/Users/jcoven/Desktop/HRS_Assembly/python/data/h16da/H16A_R.da'
dct_file = '/Users/jcoven/Desktop/HRS_Assembly/python/data/h16sta/H16A_R.dct'
read_stata_files(filename, dct_file)

,hhid,pn,psubhh,osubhh,ppn_sp,pcsr,pfamr,pfinr,pa500,pa501,...,pa065,pa066,pa068m,pa099,pa100,pa101,pa106,pa113,pvdate,pversion
0,100010,100,0.0,NaN,1,1,1,0,52,162,...,NaN,0,0,0,0,0,31,NaN,NaN,NaN
1,100030,300,0.0,NaN,1,1,1,0,32,162,...,NaN,0,0,0,0,0,21,NaN,NaN,NaN
2,100040,400,0.0,NaN,1,1,1,0,52,162,...,NaN,0,20,20,20,40,31,NaN,NaN,NaN
3,100130,401,1.0,NaN,1,1,1,0,32,162,...,NaN,0,10,10,10,20,21,NaN,NaN,NaN
4,100380,100,0.0,0.0,405,5,1,0,42,162,...,NaN,0,20,20,20,40,21,NaN,NaN,NaN
5,100380,400,0.0,0.0,101,1,5,0,42,162,...,NaN,0,20,20,20,40,21,NaN,NaN,NaN
6,100500,100,0.0,NaN,1,1,1,1,22,162,...,NaN,0,0,0,0,0,81,NaN,NaN,NaN
7,100590,200,0.0,0.0,301,1,5,0,62,162,...,NaN,0,30,30,30,60,41,NaN,NaN,NaN
8,100590,300,0.0,0.0,205,5,1,1,22,162,...,NaN,0,30,30,30,60,91,NaN,NaN,NaN
9,100630,100,0.0,NaN,1,1,1,0,42,172,...,80.0,2000,10,10,10,21,1,NaN,NaN,NaN
